# pyEGAF

Python package for analysis and manipulation of thermal neutron-capture $(n,\gamma)$ data from the Evaluated Gamma-ray Activation File (EGAF).

In [ ]:
# Import package and load data
import pyEGAF as egaf
e = egaf.EGAF()
edata = e.load_egaf()

### The $(n,\gamma)$ data sets in EGAF

In [ ]:
# EGAF targets
t = e.egaf_target_list(edata)
print(t)
print("Number of EGAF (n,g) targets = {0}".format(len(t)))

In [ ]:
# Compound-nucleus residuals
r = e.egaf_residual_list(edata)
print(r)
print("Number of CN residuals = {0}".format(len(r)))

In [ ]:
# Residual (A+1) - target (A) systems 
EGAF_Z = []
EGAF_A_target = []
EGAF_A_residual = []
rt = e.egaf_target_residual_dict(edata)
for (key, value) in rt.items(): 
    print(key, ':', value[0], value[1], value[2])
    EGAF_Z.append(int(value[1]))
    EGAF_A_target.append(int(value[2])-1)
    EGAF_A_residual.append(int(value[2]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
%matplotlib 

fig, ax = plt.subplots(figsize=(9,6))

ax.scatter(np.array(EGAF_A_target), np.array(EGAF_Z), color='k', marker='s', label=r'EGAF $(n,\gamma)$ targets')
ax.scatter(np.array(EGAF_A_residual), np.array(EGAF_Z), color='r', marker='s', alpha=0.3, label=r'EGAF $(n,\gamma)$ residuals')

ax.set_xlabel(r'$A$', size=20)
ax.set_ylabel(r'$Z$', size=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.legend(loc='best', fontsize=20)

#plt.grid()
plt.tight_layout()
plt.savefig("EGAF_nuclides.pdf", dpi=fig.dpi)
plt.show()

### A few stats from the decay scheme of the residual compound nucleus

In [ ]:
# Number of primaries
e.num_primaries(edata,"Cl36")

In [ ]:
# Number of secondaries
e.num_secondaries(edata,"Cl36")

In [ ]:
# Total number of gammas
e.num_gammas(edata,"Cl36")

In [ ]:
# Total number of levels
e.num_levels(edata,"Cl36")

In [ ]:
# Function to suppress output if desired
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    """Suppress output to console"""
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
# Create a DataFrame of all targets and residuals in EGAF together with their corresponding 
# number of primary and secondary gammas

import numpy as np
import pandas as pd

gamma_info = np.array([(r,t[0]) for (r,t) in rt.items()])
res = gamma_info[:,0]
targ = gamma_info[:,1]
with suppress_stdout():
    pri = np.array([e.num_primaries(edata,r) for r in res]).astype(int)
    sec = np.array([e.num_secondaries(edata,r) for r in res]).astype(int)
    
df = pd.DataFrame({'Target': targ, 'Residual': res, 'Primaries': pri, 'Secondaries': sec})
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)
df

### Separation energies

In [ ]:
# Proton-separation energy (AME2020)
e.get_residual_Sp_AME(edata,"Ag108")

In [ ]:
# Neutron-separation energy (AME2020)
e.get_residual_Sn_AME(edata,14,29)

In [ ]:
# Neutron-separation energy (EGAF)
e.get_residual_Sn_EGAF(edata,13,28)

In [ ]:
# All separation energies
e.get_all_separation_energies(edata,"proton")

### Inspect the RIPL file

In [ ]:
ripl=e.get_ripl(edata,"O17")

### Dump datasets to file

In [ ]:
# Call the methods below to dump O16(n,g)17 dataset to preferred file format:

# RIPL:
ripl_dump = e.get_ripl(edata,"O17",True)

# ENSDF:
ensdf_dump = e.get_ensdf(edata,"O17",True)

# JSON
json_dump = e.get_json(edata,"O17",True)

### Decay-scheme properties

In [ ]:
# Level energies and spins
e.get_residual_levels(edata,"Na24")

In [ ]:
# Find levels that have multiple spin-parity permutations
e.find_multiple_jpi(edata,"Na24")

In [ ]:
# Find levels that have unique spin-parity assignments
e.find_unique_jpi(edata,"Na24")

In [ ]:
# Find all isomeric levels in the decay scheme
e.find_isomers(edata,"Na24",units='best')

### Visualization of $\gamma$ data

In [ ]:
# Define residual nucleus
residual_cn = "Y90"
# Find corresponding target nucleus
target_ng = str([t[0] for (r,t) in rt.items() if r == residual_cn][0])
print("(n,g) target nucleus: %s"%target_ng)

In [ ]:
# Define preferred intensity output: "isotopic", "elemental", "population", "relative" - see docstrings
intensity = "isotopic"

In [ ]:
# Create an array of gamma-ray data
spe = e.get_gammas(edata, residual_cn, intensity='%s'%intensity)

gamma_energy = spe[:,4]
gamma_intensity = spe[:,6]
d_gamma_intensity = spe[:,7]

spe

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.bar(gamma_energy, gamma_intensity, width=12, color='r', label=r'%s($n,\gamma$)'%target_ng)

ax.set_yscale('log')
ax.set_xlabel(r'$E_{\gamma}$ [keV]', size=20)
if intensity == "population":
    ax.set_ylabel(r'$P$', size=20)
elif intensity == "relative":
    ax.set_ylabel(r'$I_{\gamma}$ [arb.]', size=20)
else:
    ax.set_ylabel(r'$\sigma_{\gamma}$ [b]', size=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.legend(loc='best', fontsize=20)

plt.grid()
plt.tight_layout()
plt.savefig("spectrum_{0}_ng_{1}_{2}.png".format(target_ng, residual_cn, intensity), dpi=fig.dpi)
plt.show()

### Find $\gamma$ rays in the EGAF database

In [ ]:
# Provide energy and specify preferred intensity
e.find_gamma(edata, 450, intensity='elemental')

In [ ]:
# Tune the tolerance
e.find_gamma(edata, 450, 0.1, intensity='population')

In a forensics application, and to help pin down possible candidate isotopes, we would also expect to see the strongest gammas in those isotopes


In [ ]:
# Find the strongest gammas produced in 77Se(n,g)78Se
e.get_strongest_gammas(edata, "Se78", intensity="relative")

In [ ]:
# Find the strongest gammas produced in 167Er(n,g)168Er
e.get_strongest_gammas(edata, "Er168", intensity="relative")

### CapGam-style output

pyEGAF module has functions to display data CapGam style cf. https://www.nndc.bnl.gov/capgam/

In [ ]:
# Create a plotting function and adjust according to preference
def plot_df_data(x,y,nuc):
    """Plotting method to render data CapGam style."""
    import re
    %matplotlib notebook
    
    element = str(re.findall('[A-Za-z]+', nuc)[0])
    compound_mass = int(re.findall('[0-9]+', nuc)[0])
    target_mass = compound_mass - 1
    target = str(target_mass)+element
    compound = str(compound_mass)+element
    
    plt.subplots(figsize=(8,5))
    #plt.bar(df['E'], df['RI'], width=10, color='r')
    plt.bar(x, y, width=10, color='r', label=r'$^{%i}{%s}(n,\gamma)^{%i}{%s}$'%
            (target_mass,element,compound_mass,element))
    plt.gca().set_yscale('log')
    plt.legend(loc='best', fontsize=15)
    
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel(r'$E_{\gamma}$ [keV]', size=20)
    plt.ylabel(r'$I_{\gamma}$ [%]', size=20)

    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
# Define compound nucleus as a string, e.g., 12C(n,g)13C:
compound_nucleus = 'C13'

In [ ]:
# Extract capture-gamma data for defined compound nucleus and write results to CSV file in pwd
df=e.capgam(edata,"{0}".format(compound_nucleus),"more")
df.to_csv("capgam_style_{0}.csv".format(compound_nucleus), index=False)
print(df.to_string(index=False))

In [ ]:
# Plot gamma-ray intensities from DataFrame
plot_df_data(df['E'], df['RI'], "{0}".format(compound_nucleus))

In [ ]:
# Or if you want to just take a quick look at the data, pass the compound nucleus of interest as a string
e.capgam(edata,"Si29")

In [ ]:
# Show the additional levels output
e.capgam(edata,"Si29","more")

### Adopted total thermal neutron-capture cross sections $\sigma_{0}$ in EGAF

\begin{equation}
        \nonumber
        \sigma_{0} = \sum\limits_{k=1}^{P}\sigma_{\gamma_{k}}^{\text{primary}} = \sum\limits_{i=1}^{N} \sigma_{\gamma_{i0}}^{\text{expt}}(1+\alpha_{i0}) + \sum\limits_{j=1}^{M} \sigma_{\gamma_{j0}}^{\text{sim}}.
      \end{equation}

In [ ]:
# Pass target nucleus as string to find total thermal-capture cross section
e.get_total_cross_section(edata, "Cl35")

In [ ]:
# Pass corresponding (n,g) compound nucleus as string to find sum of primary cross sections
e.sum_primaries(edata,"Cl36",intensity='isotopic')

In [ ]:
# Calculate sum of gs-feeding cross sections
e.sum_feeding_gs(edata,True,"Al28",intensity="isotopic")

In [ ]:
# Get total neutron-capture cross sections for all (n,g) targets
e.get_all_total_cross_sections(edata)

In [ ]:
# Extract isotopic abundance of (n,g) target
e.get_abundance(edata,"Cl35")

In [ ]:
e.get_abundance(edata,"Cl37")

In [ ]:
# Get natural abundances for all (n,g) targets
e.get_all_abundances(edata)

In [ ]:
# Extract total primary cross sections and total gs-feeding cross sections for all isotopes
import re
all_tcs = e.get_all_total_cross_sections(edata)

all_p = [] 
all_gs = []
A = re.compile(r'\d+')
for (k,v) in all_tcs.items():
    target = k
    residual = v[0]
    mass = int(A.findall(target)[0])
    sigma_0 = v[1]
    d_sigma_0 = v[2]
    
    with suppress_stdout():
        # Find sum of primary-gamma cross sections for each residual
        sum_p = e.sum_primaries(edata, residual, intensity='isotopic')
        
        if sum_p != None:    
            if sigma_0 > 0 and sum_p[0] > 0:
                ratio_p = sum_p[0]/sigma_0
                # For a clean plot, only including ratios less than 1:
                if ratio_p <= 1.0:
                    d_ratio_p = ratio_p * np.sqrt((d_sigma_0/sigma_0)**2 + (sum_p[1]/sum_p[0])**2)
                    #print(target, residual, mass, sigma_0, d_sigma_0, sum_p[0], sum_p[1], ratio_p, d_ratio_p)
                    all_p.append([mass, ratio_p, d_ratio_p])
        
        
        # Find sum of gs-feeding cross sections for each residual
        sum_gs = e.sum_feeding_gs(edata, True, residual, intensity="isotopic")
        
        if sum_gs != None:
            if sigma_0 > 0 and sum_gs[0] > 0:
                sim_gs = sigma_0 - sum_gs[0]
                # For a clean plot, only including positive differences:
                if sim_gs > 0:
                    d_sim_gs = np.sqrt(d_sigma_0**2 + sum_gs[1]**2)
                    ratio_missing = sim_gs/sigma_0
                    d_ratio_missing = ratio_missing * np.sqrt((d_sigma_0/sigma_0)**2 + (d_sim_gs/sim_gs)**2)
                    all_gs.append([mass, sim_gs, d_sim_gs, ratio_missing, d_ratio_missing])

In [ ]:
# Cross sections from primaries
all_p = np.array(all_p)
mass_p = all_p[:,0].astype(int)
sigma_ratio = all_p[:,1].astype(float)
d_sigma_ratio = all_p[:,2].astype(float)

# Cross sections from gs-feeding
all_gs = np.array(all_gs)
mass_gs = all_gs[:,0].astype(int)
missing_fraction = all_gs[:,3].astype(float)
d_missing_fraction = all_gs[:,4].astype(float)

In [ ]:
# Plot missing fraction from expected "simulated" contribution
%matplotlib notebook
fig, ax = plt.subplots(figsize=(9,6))

ax.errorbar(mass_gs, missing_fraction, yerr=d_missing_fraction, color='k', fmt='o', capsize=5)
ax.axhline(1.0, color='r')
ax.set_xlabel(r'$A$', size=20)
ax.set_ylabel(r'$\frac{\sum\sigma_{\gamma}^{sim}}{\sigma_{0}}$', size=20)

plt.grid()
plt.tight_layout()
plt.savefig("missing_fraction.pdf", dpi=fig.dpi)
plt.show()

In [ ]:
# Alternatively, plot sum of primaries to cross section ratio to assess completeness of primary-gamma spectrum
%matplotlib notebook
fig, ax = plt.subplots(figsize=(9,6))

ax.errorbar(mass_p, sigma_ratio, yerr=d_sigma_ratio, color='k', fmt='o', capsize=5)
ax.axhline(1.0, color='r')
ax.set_xlabel(r'$A$', size=20)
ax.set_ylabel(r'$\frac{\sum\sigma_{\gamma}^{expt}}{\sigma_{0}}$', size=20)

plt.grid()
plt.tight_layout()
plt.savefig("completeness_ratio.pdf", dpi=fig.dpi)
plt.show()

In [ ]:
# Generate CSV output for summation cross sections and compare to adopted data in EGAF

gamma_info = np.array([(r,t[0],t[1],t[2]) for (r,t) in rt.items()])
res = gamma_info[:,0]
targ = gamma_info[:,1]
Z_CN = gamma_info[:,2]
A_CN = gamma_info[:,3]

tot = []
pri = []
sec = []
sum_p = []
d_sum_p = []
sum_gs = []
d_sum_gs = []
sigma_0 = []
d_sigma_0 = []
keynumber = []
with suppress_stdout():
    tot = np.array([e.num_gammas(edata,r) for r in res]).astype(int)
    pri = np.array([e.num_primaries(edata,r) for r in res]).astype(int)
    sec = np.array([e.num_secondaries(edata,r) for r in res]).astype(int)

    sum_p = np.array([e.sum_primaries(edata, r, intensity='isotopic')[0] if e.sum_primaries(edata, r, intensity='isotopic')!=None else 0 for r in res]).astype(float)
    d_sum_p = np.array([e.sum_primaries(edata, r, intensity='isotopic')[1] if e.sum_primaries(edata, r, intensity='isotopic')!=None else 0 for r in res]).astype(float)
    sum_gs = np.array([e.sum_feeding_gs(edata, True, r, intensity='isotopic')[0] if e.sum_feeding_gs(edata, True, r, intensity='isotopic')!=None else 0 for r in res]).astype(float)
    d_sum_gs = np.array([e.sum_feeding_gs(edata, True, r, intensity='isotopic')[1] if e.sum_feeding_gs(edata, True, r, intensity='isotopic')!=None else 0 for r in res]).astype(float)

    sigma_0 = np.array([e.get_total_cross_section(edata, t)[0] for t in targ]).astype(float)
    d_sigma_0 = np.array([e.get_total_cross_section(edata, t)[1] for t in targ]).astype(float)
    keynumber = np.array([e.get_total_cross_section(edata, t)[3] for t in targ])
    
df = pd.DataFrame({'Target': targ, 'Residual': res, 'Z': Z_CN, 'A+1': A_CN, 'Total g': tot, \
                   'Primaries': pri, 'Secondaries': sec, 'Sum g(P)': sum_p, 'd_Sum g(P)': d_sum_p, \
                   'Sum g(GS)': sum_gs, 'd_Sum g(GS)': d_sum_gs,'sigma_0': sigma_0, 'd_sigma_0': d_sigma_0, \
                   'Ref': keynumber})
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)

# Write DataFrame to CSV:
df.to_csv("egaf_stats.csv", sep=",", encoding="utf-8", index=False)
df